# Monte-Carlo FrozenLake
This section explores the Monte Carlo method for reinforcement learning




## Background Theory

## Code

In [3]:
import numpy as np
import gymnasium as gym

# Initialize the environment
env = gym.make("FrozenLake-v1", map_name="4x4", is_slippery=True, render_mode = "ansi")  # is_slippery=True for stochasticity
env.reset()

state = 0; t = 0
# Simulate several steps by following a random policy
for i in range(100):
    action = env.action_space.sample()
    next_state, reward, term, trunc, info = env.step(action)
    print(f"Time {t:2d}  |  s_t {state:3d}  |  a_t {action:2d}  |  s_t+1 {next_state:3d}  |  reward {reward:.2f}  |  terminated {term:2}  |  {info}")
    # print(env.render())
    state = next_state; t += 1
    if term or trunc:
        break

Time  0  |  s_t   0  |  a_t  1  |  s_t+1   1  |  reward 0.00  |  terminated  0  |  {'prob': 0.3333333333333333}
Time  1  |  s_t   1  |  a_t  3  |  s_t+1   1  |  reward 0.00  |  terminated  0  |  {'prob': 0.3333333333333333}
Time  2  |  s_t   1  |  a_t  0  |  s_t+1   1  |  reward 0.00  |  terminated  0  |  {'prob': 0.3333333333333333}
Time  3  |  s_t   1  |  a_t  1  |  s_t+1   5  |  reward 0.00  |  terminated  1  |  {'prob': 0.3333333333333333}


In [36]:
state_size = env.observation_space.n
action_size = env.action_space.n
q_table = np.zeros((state_size,action_size))

In Monte Carlo RL, the agent is supposed to traverse entire episode(s) to observe the trajectory and reward. The return of each state is monitored and used for updating the state-action value $Q(S_t,A_t)$

Need to implement the following functionalities:
1. A function to play out one episode (trajectory) that makes the agent steps through the environemtn until a termination condition is reached. These trajectories are returned as three lists: 1. states, 2. rewards, and 3. actions
2. A function to calculate the cumulated return $G_t$ when given a list of state and a list of rewards along a trajectory generated by the function above. 
3. Need an array to store the tabular Q-value mapping from state-action $(s_t, a_t)$ to Q-value - $Q(s_t,a_t)$
4. (optional) Need an array to store the occurences of all state-action combinations - $N(s_t,a_t)$. This can however be substituted with a constant $\alpha$




### Question to self
1. How can I specify a policy to control the agent? Thus far env.step() randomly chooses the next action for the agent.
- Earlier on, we randomly sample the action from the action space of the environment. This action was then used in the env.step() function to guide the agent through one step. 
- With MC control, we gradually update the Q-value of all the state-action pairs and choose the action according to an epsilon-greedy policy

2. How can I visualize the state-action value on the environment to visualize the policy?

3. When collecting the trajectories for Monte Carlo training (for lack of better terminology), do we use epsilon-greedy in some way or do we use greedy policy?
- Initially, we start with some random policy and turn this into an epsilon-greedy policy to sample/roll out/obtain trajectories.
- The reward from these trajectories are turned into the return at each <s,a> state-action pair. These returns are normalized over the number of occurences of <s,a> and is used as an estimate of the action value function $Q(s,a)$

2. Are there alternatives to epsilon-greedy policy?


In [5]:
def get_returns(rewards, gamma = 0.9):
    ''' Function to calculate the return of each time step when given a list of rewards 
    
    For each step of the trajectory (of length T):
    - Extract the rewards from that step onward
    - Each step is multiplied by the corresponding gamma ^ index 
        the first reward received from leaving the state is not discounted
        the last reward received from the trajectory is discouned by gamma ^ (T-1)
    - Sum these values together to obtain the return at each step
    '''
    returns = np.zeros(len(rewards))
    
    for step, _ in enumerate(rewards):
        step_reward = rewards[step:]            # reward from the current step onward

        # List of discounted rewards at each time step
        return_val = [gamma ** i * step_reward[i] for i in range(len(step_reward))]
        return_val = sum(return_val)
        
        returns[step] = return_val
    return returns

In [ ]:
def get_one_episode(stochastic_policy):
    ''' This function returns a full trajectory of the agent
    Episode end conditions:
    - the agent falls into the ice or the agent reaches the reward
    - truncated when hitting a time limit

    Inputs:
    - stochastic policy - the stochastic epsilon-greedy policy (array of size action_space x states) 
    '''
    states = []
    rewards = []
    actions = []

    initial_state, _ = env.reset()
    states.append(initial_state)

    term = False
    trunc = False

    while (not term) and (not trunc):
        action = np.random.choice(env.action_space.n,
                                  p = stochastic_policy[:,state])
        next_state, reward, term, trunc, _ = env.step(action)
        
        rewards.append(reward)
        actions.append(action)
        states.append(next_state)


    return states, rewards, actions

    

In [ ]:
def deterministic_to_stochastic_policy(deterministic_policy, epsilon=0.3):
    ''' Function to convert a deterministic greedy policy to a stochastic epsilon-greedy policy 
    
    A greedy policy selects the action a_t from the state s_t that maximizes Q(s_t,a_t). 
    On the other hand, epsilon-greedy lets the agent explore the environment by following the optimal
    policy (at the time) with 1-epsilon probability, while taking other actions with epsilon probability

    Given an action space of size a, this function converts from a deterministic policy to a stochastic policy
    that follows the optimal action with 1-epsilon probability, and the other (a-1) actions randomly with epsilon probability
    
    Inputs:
    - deterministic_policy - the greedy policy, an array of size 1 x state
    - epsilon - the probability at which the agent take sub-optimal actions randomly to explore
    '''

    num_action = 4          # TO-DO: Improve to avoid hard-coding
    num_state = len(deterministic_policy)

    # Initialize the stochastic policy with all epsilon / 3
    stochastic_policy = np.full((num_action, num_state), epsilon / (num_action - 1))
    for state, action in enumerate(deterministic_policy):
        stochastic_policy[action][state] = 1 - epsilon

    return stochastic_policy


In [ ]:
def update_MC_policy(stochastic_policy, n_episodes = 100, gamma = 0.9):
    ''' This function samples trajectories under a specified stochastic policy to approximate the value function 
    
    (Value function evaluation from multiple trajectories)

    Inputs:
    - stochastic_policy - the policy that the agent follows to roll out trajectories. A matrix of size num_actions x num_states
    - n_episodes - the number of trajectories sampled over which we find the average return G_t (to approximate the true action value function)
    
    Outputs:
    - policy - the deterministic greedy policy (the action in each state that maximizes the action value function)
    - state_action_values - array of size (n_actions x n_states)
    '''

    n_actions = stochastic_policy.shape[0]
    n_states = stochastic_policy.shape[0]

    state_action_return_total = np.zeros((n_actions, n_states))
    state_action_count = np.zeros((n_actions, n_states))

    for episode in range(n_episodes):
        # Get one full episode
        states, rewards, actions = get_one_episode(stochastic_policy)

        # Calculate the return
        returns = get_returns(rewards, gamma)

        # Update the total return for each state-action pair in the trajectory
        for idx, _ in enumerate(states[:-1]):       # iterate through the trajectory (terminal state excluded)
            state_action_return_total[actions[idx],states[idx]] += returns[idx]
            state_action_count[actions[idx],states[idx]] += 1

    # Update the state-action value function by dividing the total state
    state_action_values = state_action_return_total / (state_action_count + 0.0001)

    # Deterministic greedy policy
    policy = np.argmax(state_action_values, axis=0)

    return policy, state_action_values

[0. 0.]
[0, 4, 5]
[0.0, 0.0]


Now that we have a function to perform value function evaluation, we need to somehow incorporate this improved evaluation into the improvement of the control policy. The output of policy evaluation are: 1. the state-action values $Q(s,a)$ $\forall s \in \mathcal{S}, \, a \in \mathcal{A}$, and 2. the deterministic greedy policy.

We will use this new deterministic greedy policy for building out trajectories for the next policy evaluation process
